## Example loading Landsat and Sentinel-2

In [ ]:
import datacube
from pyproj import Transformer
import matplotlib.pyplot as plt

### Landsat products:

- ga_ls5t_ard_3 (1986 - 2011) [decommissioned]
- ga_ls7e_ard_3 (1999 - 2022) [decommissioned]
- ga_ls8c_ard_3 (2013 - current)
- ga_ls9c_ard_3 (2021 - current)

### Sentinel-2 products:

- ga_s2am_ard_3 (2015 - 2025) [decommissioned]
- ga_s2bm_ard_3 (2017 - current)
- ga_s2cm_ard_3 (2025 - current)

In [ ]:
# Initialise datacube

dc = datacube.Datacube()

In [ ]:
# Transform search coordinates from EPSG:3577 to EPSG:4326
# (Datacube searches are typically done in EPGS:4326)

# Define transformer from your CRS to WGS84
transformer = Transformer.from_crs("EPSG:3577", "EPSG:4326", always_xy=True)

# Example: reproject bounding box from EPSG:28355 to EPSG:4326
# (Central NSW)
x_min, x_max = 1300000, 1305000
y_min, y_max = -3700000, -3705000

lon_min, lat_min = transformer.transform(x_min, y_min)
lon_max, lat_max = transformer.transform(x_max, y_max)

In [ ]:
# Define search criteria (time and space)

longitude = (lon_min, lon_max)
latitude = (lat_min, lat_max)
time = ("2024-01-01", "2024-01-07")

In [ ]:
# Find datasets

datasets = dc.find_datasets(
    product="ga_ls8c_ard_3",
    longitude=longitude,
    latitude=latitude,
    time=time
)

In [ ]:
print(f"Number of datasets found: {len(datasets)}")

In [ ]:
datasets

In [ ]:
# Load datasets (lazy)

product = "ga_ls8c_ard_3"  # landsat 8
measurements = ["nbart_red", "nbart_blue"]
output_crs = "EPSG:3577"
resolution = [-30, 30]

dask_chunks = {
    "time": 1,
    "y": 500,
    "x": 500
}

ds = dc.load(product=product,
             measurements=measurements,
             #datasets=datasets,  # Can pass in dataset/s instead of x,y,time if you want the whole tile instead.
             crs="EPSG:3577",
             x=(x_min, x_max),
             y=(y_min, y_max),
             time=time,
             output_crs=output_crs,
             resolution=resolution,
             dask_chunks=dask_chunks,
             skip_broken_datasets=True  # Important!
             )

In [ ]:
ds

In [ ]:
# Compute (load data into memory)

ds = ds.compute()

In [ ]:
ds

In [ ]:
# Visualise dataset

band = ds["nbart_red"].isel(time=0)

# Plot with xarray’s wrapper around matplotlib
band.plot.imshow(cmap="viridis")  # or cmap='gray', 'RdYlGn', etc.
plt.title("Red Band")
plt.xlabel("x")
plt.ylabel("y")
plt.show()